# Web App Event Logs - Exploratory Data Analysis

This notebook analyzes web app event logs stored as JSON files in `reports/monitor-logs/YYYY/MM/DD/` structure.

Each JSON file contains:
```json
{
  "sessionId": "session_1754052978056_4i3x2c3bc1g3g4j2",
  "timestamp": "20250801T125618127Z",
  "eventType": "sessionStart",
  "payload": { ... }
}
```

In [1]:
import glob
import json
import os
from collections import Counter, defaultdict
from pathlib import Path

import duckdb
import pandas as pd

# Set up paths
log_base_path = Path("monitor-logs")
print(f"Base log path: {log_base_path}")
print(f"Path exists: {log_base_path.exists()}")

# Initialize DuckDB in-memory database
db = duckdb.sql("CREATE OR REPLACE TABLE events AS SELECT NULL WHERE FALSE")

Base log path: monitor-logs
Path exists: True


## 1. Load All JSON Files

Find and load all JSON files from the log directory structure.

In [ ]:
def load_all_log_files(base_path="monitor-logs", min_date="20250705"):
    """
    Load all JSON files from the log directory structure.

    Handles the recursive YYYY/MM/DD/ folder structure.
    Each JSON file contains: sessionId, timestamp, eventType, payload.

    Args:
        base_path: Path to monitor-logs directory
        min_date: Only load files from this date onwards (format: YYYYMMDD, e.g., '20250705')

    """
    all_events = []
    file_count = 0
    error_count = 0
    skipped_count = 0

    # Use glob to find all JSON files recursively
    json_pattern = os.path.join(base_path, "**/*.json")
    json_files = glob.glob(json_pattern, recursive=True)

    print(f"Found {len(json_files)} JSON files total\n")
    if min_date:
        print(f"Filtering for files from {min_date} onwards (July 5, 2025)\n")

    # Load each file
    for file_path in json_files:
        # Filter by date if min_date is specified
        if min_date:
            # Extract date from path: monitor-logs/YYYY/MM/DD/...
            path_parts = file_path.split(os.sep)
            if len(path_parts) >= 4:
                try:
                    year = path_parts[-4]
                    month = path_parts[-3]
                    day = path_parts[-2]
                    file_date = f"{year}{month}{day}"
                    if file_date < min_date:
                        skipped_count += 1
                        continue
                except (ValueError, IndexError):
                    pass

        try:
            with open(file_path) as f:
                event_data = json.load(f)

            # Validate required fields
            if all(
                key in event_data for key in ["sessionId", "timestamp", "eventType"]
            ):
                all_events.append(event_data)
                file_count += 1
            else:
                error_count += 1

        except (OSError, json.JSONDecodeError):
            error_count += 1

    print(f"Successfully loaded: {file_count} files")
    print(f"Skipped (before cutoff): {skipped_count} files")
    if error_count > 0:
        print(f"Errors encountered: {error_count} files\n")

    return all_events


# Load all events (from July 5, 2025 onwards)
all_events = load_all_log_files(min_date="20250705")

print(f"\nTotal events loaded: {len(all_events)}")

Found 3965 JSON files total

Filtering for files from 20250705 onwards (July 5, 2025)

Successfully loaded: 1849 files
Skipped (before cutoff): 2116 files

Total events loaded: 1849


## 1b. Inject into DuckDB

Store events in DuckDB with native JSON support for efficient querying.

In [3]:
# Convert events to DuckDB table
# DuckDB will automatically handle JSON columns (payload, server_context)

# Create DataFrame first for easier insertion into DuckDB
events_df = pd.DataFrame(all_events)

# Parse timestamp to datetime
events_df["timestamp"] = pd.to_datetime(
    events_df["timestamp"], format="%Y%m%dT%H%M%S%fZ"
)

# Create DuckDB table from DataFrame
# DuckDB automatically infers types and keeps JSON as JSON
db = duckdb.from_df(events_df)

# Show table schema
print("DuckDB Table Schema:")
print(db.describe())

print(f"\nTable shape: {db.shape}")
print("\nFirst few rows:")
print(db.limit(5).to_df())

DuckDB Table Schema:
┌─────────┬──────────────────────────────────────────┬─────────────────────────┬──────────────────┬──────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  aggr   │                sessionId                 │        timestamp        │    eventType     │                               payload                                │                                                                server_context                                                                │
│ varchar │                 varchar                  │         varchar         │     varchar      │                               varchar                                │                                                                   varchar                                                                    │
├─────────┼──────────────────────────

## 2. Count Events and Sessions

Get key statistics on number of events and unique sessions.

In [4]:
# Extract basic counts
session_ids = set(event["sessionId"] for event in all_events)
event_types = Counter(event["eventType"] for event in all_events)

print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
print(f"Total events:          {len(all_events):,}")
print(f"Total unique sessions: {len(session_ids):,}")
print(f"Avg events per session: {len(all_events) / len(session_ids):.2f}")


SUMMARY STATISTICS
Total events:          1,849
Total unique sessions: 259
Avg events per session: 7.14


## 3. Events by Type

Breakdown of event types in the dataset.

In [5]:
print("\n" + "=" * 60)
print("EVENTS BY TYPE")
print("=" * 60)

event_type_df = pd.DataFrame(
    [
        {"eventType": event_type, "count": count}
        for event_type, count in event_types.most_common()
    ]
)

event_type_df["percentage"] = (
    event_type_df["count"] / event_type_df["count"].sum() * 100
).round(2)
event_type_df["cumulative_pct"] = event_type_df["percentage"].cumsum().round(2)

print(event_type_df.to_string(index=False))


EVENTS BY TYPE
       eventType  count  percentage  cumulative_pct
visibilityChange   1043       56.41           56.41
    sessionStart    259       14.01           70.42
         request    188       10.17           80.59
         article    129        6.98           87.57
        response    104        5.62           93.19
       userInput    103        5.57           98.76
        feedback     19        1.03           99.79
     userProfile      4        0.22          100.01


## 4. Session Statistics

Analyze events distribution across sessions.

In [6]:
# Count events per session
events_per_session = Counter(event["sessionId"] for event in all_events)

session_stats_df = pd.DataFrame(
    [
        {"sessionId": session_id, "event_count": count}
        for session_id, count in events_per_session.items()
    ]
)

print("\n" + "=" * 60)
print("SESSION EVENT DISTRIBUTION")
print("=" * 60)
print(session_stats_df["event_count"].describe())

print("\n" + "=" * 60)
print("TOP 15 SESSIONS BY EVENT COUNT")
print("=" * 60)
top_sessions = session_stats_df.nlargest(15, "event_count")
print(top_sessions.to_string(index=False))


SESSION EVENT DISTRIBUTION
count    259.000000
mean       7.138996
std       10.661638
min        1.000000
25%        1.000000
50%        2.000000
75%        8.500000
max       69.000000
Name: event_count, dtype: float64

TOP 15 SESSIONS BY EVENT COUNT
                               sessionId  event_count
 session_1752409095264_636v5ft1d1r1q1b2e           69
  session_1752134107494_f2a4x3f6ij1a6v1k           65
 session_1751905006481_d6r5o183x49463s5a           63
 session_1752137691874_6v5x52c3w29583f4u           51
session_1754242429945_5t132a3p4k2k2k6u52           50
  session_1752170012739_64e6b366d3s5dh2w           42
 session_1751898673280_4a6i353p2q5m1ug5g           38
session_1752324436901_531m2y2w226a312t3x           37
 session_1752834153230_1e6r3f5x6x6j40q6n           34
session_1759954995263_186g2v642r4q2y3f61           31
 session_1752481058246_33t2f382f3w4d2y3u           30
session_1752134176025_3x2b5j1e5n2v4l622p           28
 session_1752139634570_621c2g2e1r2x6v47y    

## 5. Sample Events

Peek at sample events from each type to understand data structure.

In [7]:
# Group events by type and show a sample from each
events_by_type = defaultdict(list)
for event in all_events:
    events_by_type[event["eventType"]].append(event)

print("\n" + "=" * 60)
print("SAMPLE EVENTS BY TYPE")
print("=" * 60)

for event_type in sorted(events_by_type.keys()):
    sample = events_by_type[event_type][0]
    print(f"\n{event_type.upper()}:")
    print(json.dumps(sample, indent=2, default=str)[:600])
    print("-" * 60)


SAMPLE EVENTS BY TYPE

ARTICLE:
{
  "sessionId": "session_1752137723364_6y2xy3v6v1n72261j",
  "timestamp": "20250710T093139577Z",
  "eventType": "article",
  "payload": {
    "queryId": "query_1752139891296_usat982",
    "htmlContent": "<h2>Table des centrales \u00e9lectriques en tunisie ?</h2>\n            <div class=\"attribution\">Generated by Cirdi on Thursday, July 10, 2025</div>\n            <div class=\"generation-stats\"><span> Tokens: 0 in, 1,088 out. LLM cost: 0.87 cent. Self hosting cost: 0.20 cent (H100 VPS hourly rate for 2.37s.). </span></div>\n            <div class=\"settings-stats\">Retrieval en mode vanilla
------------------------------------------------------------

FEEDBACK:
{
  "sessionId": "session_1752134107494_f2a4x3f6ij1a6v1k",
  "timestamp": "20250710T090822606Z",
  "eventType": "feedback",
  "payload": {
    "type": "up",
    "comment": ""
  },
  "server_context": {
    "client_ip": "193.51.112.3",
    "forwarded_for": "193.51.112.3",
    "received_at": "20

## 6. Create DataFrame for Further Analysis

Convert to pandas DataFrame with parsed timestamps.

In [8]:
# Create DataFrame
events_df = pd.DataFrame(all_events)

# Parse timestamp to datetime
events_df["timestamp"] = pd.to_datetime(
    events_df["timestamp"], format="%Y%m%dT%H%M%S%fZ"
)

print(f"\nDataFrame shape: {events_df.shape}")
print(f"\nColumns: {events_df.columns.tolist()}")
print("\nData types:")
print(events_df.dtypes)
print("\nFirst few rows:")
print(events_df[["sessionId", "timestamp", "eventType"]].head(10))


DataFrame shape: (1849, 5)

Columns: ['sessionId', 'timestamp', 'eventType', 'payload', 'server_context']

Data types:
sessionId                 object
timestamp         datetime64[ns]
eventType                 object
payload                   object
server_context            object
dtype: object

First few rows:
                                  sessionId               timestamp  \
0  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 07:56:16.048   
1  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 07:56:23.775   
2  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 07:56:41.475   
3  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 08:11:41.024   
4  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 08:25:32.137   
5  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 08:25:37.143   
6  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 08:25:57.632   
7  session_1752134176025_3x2b5j1e5n2v4l622p 2025-07-10 08:25:59.631   
8  session_1752134176025_3x2b5j1e5n2v4l622p 2

## DuckDB Query Examples

Leverage DuckDB's JSON and SQL capabilities for powerful analysis.

In [ ]:
# Example 1: Extract nested payload fields
print("\n" + "=" * 60)
print("PAYLOAD STRUCTURE BY EVENT TYPE")
print("=" * 60)

query1 = """
SELECT
    eventType,
    COUNT(*) as count,
    json_keys(payload) as payload_keys
FROM db
GROUP BY eventType, json_keys(payload)
ORDER BY count DESC
LIMIT 10
"""

result1 = duckdb.sql(query1).to_df()
print(result1)

# Example 2: Extract specific JSON fields
print("\n" + "=" * 60)
print("SESSION START INFO")
print("=" * 60)

query2 = """
SELECT
    sessionId,
    timestamp,
    json_extract_string(payload, '$.userAgent') as userAgent,
    json_extract_string(payload, '$.language') as language
FROM db
WHERE eventType = 'sessionStart'
LIMIT 5
"""

result2 = duckdb.sql(query2).to_df()
print(result2)

# Example 3: Count events by type and session
print("\n" + "=" * 60)
print("EVENT COUNTS BY SESSION AND TYPE")
print("=" * 60)

query3 = """
SELECT
    sessionId,
    eventType,
    COUNT(*) as event_count
FROM db
GROUP BY sessionId, eventType
ORDER BY sessionId, event_count DESC
LIMIT 20
"""

result3 = duckdb.sql(query3).to_df()
print(result3)


PAYLOAD STRUCTURE BY EVENT TYPE


InvalidInputException: Invalid Input Error: Malformed JSON at byte 1 of input: unexpected character.  Input: "{'sessionId': 'session_1752134176025_3x2b5j1e5n..."